# Headline Classification using Stacking Ensemble method

### Importing libraries needed

In [ ]:
#------------------------------------------Libraries---------------------------------------------------------------#
####################################################################################################################
#-------------------------------------Boiler Plate Imports---------------------------------------------------------#
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#---------------------------------------Text Processing------------------------------------------------------------#
import regex
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
#------------------------------------Metrics and Validation---------------------------------------------------------#
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
#-------------------------------------Models to be trained----------------------------------------------------------#
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import xgboost
#####################################################################################################################

## Loading Data

In [ ]:
train = pd.read_csv('/kaggle/input/ireland-historical-news/ireland-news-headlines.csv')
test = pd.read_csv('/kaggle/input/ireland-historical-news/w3-latnigrin-tokens.csv')

## Peek Training Data

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
train.headline_category.value_counts()

* We could clearly make out that for every category there are multiple sub-categories which only increases the complexity of our predictions and increases variance as the sub-categories are less in number. 

* One way to tackle this, is to merge the sub-categories into their respective 'super class'. This wouldn't effect the predictions much as the sub-categories are clearly out-numbered.

In [ ]:
year = [] 
month = [] 
day = [] 

dates = train.publish_date.values

for date in dates:
    str_date = list(str(date))
    year.append(int("".join(str_date[0:4]))) 
    month.append(int("".join(str_date[4:6])))
    day.append(int("".join(str_date[6:8])))

In [ ]:
train['year'] = year
train['month'] = month
train['day'] = day

train.drop(['publish_date'] , axis=1,inplace=True) 
train = train[train['headline_category'] != 'removed']
train.head()

In [ ]:
print('Unique Headlines Categories: {}'.format(len(train.headline_category.unique())))

In [ ]:
set([category for category in train.headline_category if "." not in category] ) 

In [ ]:
train['headline_category'] = train.headline_category.apply(lambda x: x.split(".")[0]) 
train = train.loc[train.headline_category != 'removed']

## Clean the textual data

* Normalize the data by converting to lower case.
* Remove punctuations, stopwords and special characters.
* Tokenize the text, i.e segment the text into sentences and further into words.
* Extract the root word by 'Lemmatizing' each word. Ex: Lemmatize('swimming') = 'swim'

**Helper Function For Data Preparation**

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    filtered_text = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha()]
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in filtered_text if not w in stop_words] 
    return " ".join(filtered_text)

In [ ]:
train["filtered_text"] = train.headline_text.apply(lambda x : filter_text(x, stop)) 
train.head()

## Exploring Training Data

### Basic Visualization

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(train.headline_category, palette = sns.color_palette("mako"))

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.lineplot(x=train.year.value_counts().index.values,y=train.year.value_counts().values, color = 'seagreen')
ax = plt.title('Number of Published News by Year')

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.lineplot(x=train.month.value_counts().index.values,y=train.month.value_counts().values, color = 'seagreen')
ax = plt.title('Number of Published News by Month')

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.lineplot(x=train.day.value_counts().index.values,y=train.day.value_counts().values, color = 'seagreen')
ax = plt.title('Number of Published News by Day')

### Word Clouds

**Helper Function for word clouds**

In [ ]:
def make_wordcloud(words,title):
    cloud = WordCloud(width=1920, height=1080,max_font_size=200, max_words=300, background_color="white").generate(words)
    plt.figure(figsize=(20,20))
    plt.imshow(cloud, interpolation="gaussian")
    plt.axis("off") 
    plt.title(title, fontsize=60)
    plt.show()

In [ ]:
all_text = " ".join(train[train.headline_category == "news"].filtered_text) 
make_wordcloud(all_text, "News")

In [ ]:
all_text = " ".join(train[train.headline_category == "culture"].filtered_text) 
make_wordcloud(all_text, "Culture")

In [ ]:
all_text = " ".join(train[train.headline_category == "opinion"].filtered_text) 
make_wordcloud(all_text, "Opinion")

In [ ]:
all_text = " ".join(train[train.headline_category == "business"].filtered_text) 
make_wordcloud(all_text, "Business")

In [ ]:
all_text = " ".join(train[train.headline_category == "sport"].filtered_text) 
make_wordcloud(all_text, "Sport")

In [ ]:
all_text = " ".join(train[train.headline_category == "lifestyle"].filtered_text) 
make_wordcloud(all_text, "Lifestyle")

## Vectorizing Cleaned Text For Modelling

In [ ]:
tfidf = TfidfVectorizer(lowercase=False)
train_vec = tfidf.fit_transform(train['filtered_text'])

In [ ]:
train_vec.shape

### Splitting into training and validation sets

In [ ]:
train['classification'] = train['headline_category'].replace(['news','culture','opinion','business','sport','lifestyle'],[0,1,2,3,4,5])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_vec,train['classification'], stratify=train['classification'], test_size=0.2)

## Modelling

### Base Models

* Logistic Regression with RandomizedSearchCV.
* Multinomial Naive Bayes with RandomizedSearchCV.
* SDGClassifier with Squared Hinge Loss -> SVM.

### Ensemble Methods

* Hard Voting Classifier.
* Stacking Classifier with XGBClassifier as the "final estimator" or "meta learner".

## Base Models

### Logistic Regression with RandomizedSearchCV

Cross-Validation takes a lot of time, typically 1hr 30min (due to the size of dataset) on an 8 core CPU for Logistic Regression.<br>
The parameters used were obtained by already performing Cross Validation.

In [ ]:
#C = np.arange(0, 1, 0.001)
#l1_ratio = np.ratio(0, 1, 0.01)
#max_iter = range(100, 500)
#warm_start = [True, False]
#solver = ['lbfgs', 'newton-cg']
#penalty = ['l2', 'l1']

#params = {
#    'C' : C,
#    'l1_ratio' : l1_ratio,
#    'max_iter' : max_iter,
#    'warm_start' : warm_start,
#    'solver' : solver,
#    'penalty' : penalty
#}
#
#random_search = RandomizedSearchCV(
#    estimator = LogisticRegression(),
#    param_distributions = params,
#    n_iter = 100,
#    cv = 3,
#    n_jobs = -1,
#    random_state = 1
#).fit(x_train, y_train)
#
#random_search.best_params_

In [ ]:
model_lr = LogisticRegression(
    C=0.98, 
    l1_ratio=0.23, 
    max_iter=430, 
    random_state=1,
    warm_start=True
).fit(x_train, y_train)

model_lr.score(x_train, y_train)

In [ ]:
predicted = model_lr.predict(x_val)

lr_acc = accuracy_score(y_val,predicted)
lr_cop = cohen_kappa_score(y_val,predicted)
lr = pd.DataFrame([lr_acc, lr_cop], columns = ['Logistic Regression with RandomizedSearchCV'])

print("Test score: {:.2f}".format(lr_acc))
print("Cohen Kappa score: {:.2f}".format(lr_cop))

plt.figure(figsize=(15,10))
ax = sns.heatmap(confusion_matrix(y_val,predicted),annot=True)
ax = ax.set(xlabel='Predicted',ylabel='True',title='Confusion Matrix',
            xticklabels=(['news','culture','opinion','business','sport','lifestyle']),
            yticklabels=(['news','culture','opinion','business','sport','lifestyle']))

### Multinomial Naive Bayes with RandomizedSearchCV
Cross-Validation takes typically 10min (due to the size of dataset) on an 8 core CPU for Multinomial Naive Bayes.<br>
The parameters used were obtained by already performing Cross Validation.

In [ ]:
#alpha = np.arange(0, 1, 0.001)
#fit_prior = [True, False]

#params = {
#    'alpha' : alpha,
#    'fit_prior' : fit_prior
#}
#
#random_search = RandomizedSearchCV(
#    estimator = MultinomialNB(),
#    param_distributions = params,
#    n_iter = 100,
#    cv = 3,
#    n_jobs = -1,
#    random_state = 1
#).fit(x_train, y_train)
#
#random_search.best_params_

In [ ]:
model_mnb = MultinomialNB(alpha=1.9000000000000001, fit_prior=False).fit(x_train, y_train)

model_mnb.score(x_train, y_train)

In [ ]:
predicted = model_mnb.predict(x_val)

mnb_acc = accuracy_score(y_val,predicted)
mnb_cop = cohen_kappa_score(y_val,predicted)
mnb = pd.DataFrame([mnb_acc, mnb_cop], columns = ['MultinomialNB with RandomizedSearchCV'])

print("Test score: {:.2f}".format(mnb_acc))
print("Cohen Kappa score: {:.2f}".format(mnb_cop))

plt.figure(figsize=(15,10))
ax = sns.heatmap(confusion_matrix(y_val,predicted),annot=True)
ax = ax.set(xlabel='Predicted',ylabel='True',title='Confusion Matrix',
            xticklabels=(['news','culture','opinion','business','sport','lifestyle']),
            yticklabels=(['news','culture','opinion','business','sport','lifestyle']))

### SGDClassifier with Squared Hinge Loss
* Support Vector Classifiers can be implemented by using the SGDClassifier with **'hinge'** as the loss function.<br><br>
* Since the **'squared hinge loss'** removes the possibility of negative class in the range, makes it the optimal choice for the loss function.<br><br>
* Cross Validation **failed** to yeild better results than the base model defined, hence skipped.

In [ ]:
model_sgd_hinge = SGDClassifier(
    loss='squared_hinge',
    penalty='l2',
    alpha=0.0001,
    l1_ratio=0.15,
    fit_intercept=True,
    max_iter=1000,
    tol=0.001,
    shuffle=True,
    verbose=0,
    epsilon=0.1,
    n_jobs=-1,
    random_state=1,
    learning_rate='optimal',
    eta0=0.0,
    power_t=0.5,
    early_stopping=False,
    validation_fraction=0.1,
    n_iter_no_change=5,
    class_weight=None,
    warm_start=False,
    average=False).fit(x_train, y_train)

model_sgd_hinge.score(x_train, y_train)

In [ ]:
predicted = model_sgd_hinge.predict(x_val)

sgd_hinge_acc = accuracy_score(y_val,predicted)
sgd_hinge_cop = cohen_kappa_score(y_val,predicted)
sgd_hinge = pd.DataFrame([sgd_hinge_acc, sgd_hinge_cop], columns = ['SGDClassifier with Squared Hinge Loss'])

print("Test score: {:.2f}".format(sgd_hinge_acc))
print("Cohen Kappa score: {:.2f}".format(sgd_hinge_cop))
plt.figure(figsize=(15,10))
ax = sns.heatmap(confusion_matrix(y_val,predicted),annot=True)
ax = ax.set(xlabel='Predicted',ylabel='True',title='Confusion Matrix',
            xticklabels=(['news','culture','opinion','business','sport','lifestyle']),
            yticklabels=(['news','culture','opinion','business','sport','lifestyle']))

## Ensemble Methods

In [ ]:
estimators = [
    ('svm', model_sgd_hinge),
    ('mnb', model_mnb),
    ('lr', model_lr)
]

estimators

### Hard Voting Classifier

In [ ]:
model_voting = VotingClassifier(
    estimators = estimators,
    voting='hard', 
    n_jobs=-1,
    flatten_transform=True, 
    verbose=1).fit(x_train, y_train)

model_voting.score(x_train, y_train)

In [ ]:
predicted = model_voting.predict(x_val)

voting_acc = accuracy_score(y_val,predicted)
voting_cop = cohen_kappa_score(y_val,predicted)
voting = pd.DataFrame([voting_acc, voting_cop], columns = ['Hard Voting Classifier'])

print("Test score: {:.2f}".format(voting_acc))
print("Cohen Kappa score: {:.2f}".format(voting_cop))

plt.figure(figsize=(15,10))
ax = sns.heatmap(confusion_matrix(y_val,predicted),annot=True)
ax = ax.set(xlabel='Predicted',ylabel='True',title='Confusion Matrix',
            xticklabels=(['news','culture','opinion','business','sport','lifestyle']),
            yticklabels=(['news','culture','opinion','business','sport','lifestyle']))

### Stacking Classifier

In [ ]:
xgc = xgboost.XGBClassifier()

model_stack = StackingClassifier(
    estimators=estimators,
    final_estimator=xgc,
    n_jobs = -1,
    verbose = 1
)

model_stack.fit(x_train, y_train)

model_stack.score(x_train, y_train)

In [ ]:
predicted = model_stack.predict(x_val)

stack_acc = accuracy_score(y_val,predicted)
stack_cop = cohen_kappa_score(y_val,predicted)
stack = pd.DataFrame([stack_acc, stack_cop], columns = ['Stacking Classifier'])

print("Test score: {:.2f}".format(stack_acc))
print("Cohen Kappa score: {:.2f}".format(stack_cop))

plt.figure(figsize=(15,10))
ax = sns.heatmap(confusion_matrix(y_val,predicted),annot=True)
ax = ax.set(xlabel='Predicted',ylabel='True',title='Confusion Matrix',
            xticklabels=(['news','culture','opinion','business','sport','lifestyle']),
            yticklabels=(['news','culture','opinion','business','sport','lifestyle']))

## Model Comparision

In [ ]:
model_comp = pd.concat([lr, mnb, sgd_hinge, voting, stack], axis = 1)
model_comp

## Predict the Test Data

In [ ]:
test_vec = tfidf.transform(test.iloc[:, -1])
test_vec.shape

In [ ]:
cat = ['news','culture','opinion','business','sport','lifestyle']
code = [0,1,2,3,4,5]
dic = dict([(code[x], cat[x])for x in range(6)])
dic

In [ ]:
pred = model_stack.predict(test_vec)

predictions = []
for i in pred:
    predictions.append(dic[i])
test['Category'] = predictions

test.head()

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(test.Category, palette = sns.color_palette("mako"))

## Inferences and Future Works

* **Though Naive Bayes based models are considered to be the go-to models for Text Classification, they couldn't perform well as the linear model because of the size of the dataset. Same goes for SVM, these models are suitable for small to medium sized datasets.**

* **Tree Based Models such as Decision Trees and other ensemble methods such as GradientBoost and AdaBoost could also be used but require much more computational power and proper hyperparameter tuning.**

* **For a huge dataset like this, traditional ML models can prove short. Hence artificial neural network architectures using LSTMs and also CNNs can be proved as a step up. They would also require an extra computational capacity with GPUs.**

## Credits : 

* [EDA and Data Preparation](https://www.kaggle.com/darkrubiks/classifying-irish-times-headlines-categories)
* [SVC implemented with SGDClassifier](https://www.kaggle.com/flubber/text-classification-irish-times-news-dataset)